In [ ]:
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import datetime as dt

from factor_momentum import assetspace_signal_monthly

from sf_quant.data import load_factors

In [ ]:
TYPE = '1m cross-section'

In [ ]:
start = dt.date(2000,1,1)
end = dt.date(2020,1,1)

monthly_signal = assetspace_signal_monthly(start=start, end=end, type=TYPE)
monthly_signal

In [ ]:
monthly_signal = (monthly_signal.with_columns(
    (pl.col('ret') * pl.col('market_cap')/pl.col('market_cap').sum().over('month')).alias('wret')
)
)

In [ ]:
benchmark = monthly_signal.group_by(['month']).agg(
    pl.col('wret').mean()
).sort('month')

benchmark

In [ ]:
binned = (monthly_signal.with_columns(
    pl.col('signal').rank('dense').over('month').alias('rank')
)
.with_columns(
    (pl.lit("p") +
    (pl.col('rank')/pl.col('rank').count().over('month')*10 - 0.0001)
    .floor()
    .cast(pl.Int32).cast(pl.Utf8))
    .alias('bin')
)
.group_by(['month', 'bin']).agg(
    pl.col('ret').mean().alias('ret'),
    pl.col('wret').count().alias('count'),
    pl.col('market_cap').sum()
)
.sort(['bin', 'month'])
.with_columns(
    (pl.col('ret') * pl.col('market_cap') / pl.col('market_cap').sum().over('month')).alias('wret')
)
)

ports_ew = (binned
.pivot(on='bin', index='month', values='ret')
.drop_nulls()
)

ports_vw = (binned
.pivot(on='bin', index='month', values='wret')
.drop_nulls()
)

ports_vw

In [ ]:
plt.style.use('default')

def plot_deciles(df, name, cum_sum=True, long=False):
    if long:
        plt.figure(figsize=(12, 3))
    else:
        plt.figure(figsize=(7, 5))
    plt.title(name)
    plt.grid(True)
    dates = df['month']

    for i in range(10):
        if cum_sum:
            plt.plot(dates, df[f'p{i}'].cum_sum(),label = f'p{i}')
        else:
            plt.plot(dates, df[f'p{i}'],label = f'p{i}')
            
    plt.legend()
    plt.show()

In [ ]:
plot_deciles(ports_ew, f"{TYPE.upper()}: Cumulative Log Returns (EW)")

In [ ]:
ports_ew = ports_ew.with_columns(
    (pl.col('p9')-pl.col('p0')).alias('spread')
)

In [ ]:
from dotenv import load_dotenv
import os 

load_dotenv()

tmp = os.getenv('TMP')
ports_ew.write_parquet(f'{tmp}/deciles.parquet')

In [ ]:
dates = ports_ew['month']

plt.figure(figsize=(7, 5))
plt.title(f'{TYPE.upper()}: Spread Portfolios')

plt.grid(True)

s9 = (ports_ew['p9'] - ports_ew['p0'])

plt.plot(dates, s9.cum_sum(), label='p9 - p0 spread')
plt.legend()


plt.show()

In [ ]:
def Sharpe(df, name):
    annual_ret =  df.mean() * 12
    annual_vol = (df).std() * np.sqrt(12)
    sharpe = annual_ret / annual_vol

    print(
        f'{name}: \n'
        f'  Annual Return: {annual_ret:.2f} | '
        f'Annual Vol: {annual_vol:.2f} | '
        f'Sharpe: {sharpe:.2f}'
    )

Sharpe(s9, 'Spread 9')
Sharpe(s5, 'Spread 5')
Sharpe(s1, 'Spread 1')

In [ ]:
plot_deciles(ports_vw, f"{TYPE.upper()}: Cumulative Log Returns (VW)")

In [ ]:
mes = (binned
.pivot(on='bin', index='month', values='market_cap')
.drop_nulls()
)

In [ ]:
plot_deciles(mes, f'{TYPE.upper()}: Market Cap Per Portfolio', False, True)

In [ ]:
dates = ports_vw['month']
counts = (binned.pivot(
    on='bin', index='month', values='count'
)
)

plt.figure(figsize=(8, 5))
plt.title("Bin Counts")

for i in range(10):
    plt.plot(dates, counts[f'p{i}'], label = f'{i}')

plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
plt.title("1 Year Bin Counts")

for i in range(10):
    plt.plot(dates[:12], counts[f'p{i}'][:12], label = f'{i}')

plt.legend()
plt.show()